In [21]:
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import HeatMapWithTime

columnas_deseadas = ['Latitud', 'Longitud', 'Fecha_Acc', 'Hora_Acc', 'Dia_Semana_Acc']
df = pd.read_csv('accidentes.csv', usecols=columnas_deseadas, sep=';')
df = df.dropna()
df['Fecha_Acc'] = pd.to_datetime(df['Fecha_Acc'], errors='coerce')

df.head();

In [25]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['Longitud'], df['Latitud']))
gdf.set_crs(epsg=4326, inplace=True)

# Crear cuadrantes de 0.01° (~1km x 1km)
gdf['cuadrante_x'] = (gdf['Longitud'] * 100).astype(int)
gdf['cuadrante_y'] = (gdf['Latitud'] * 100).astype(int)
gdf['zona'] = gdf['cuadrante_x'].astype(str) + '_' + gdf['cuadrante_y'].astype(str)

# Agrupar por zona, día y hora
gdf['datetime'] = gdf['Fecha_Acc'].dt.floor('h')
heatmap_data = gdf.groupby(['datetime']).apply(
    lambda x: [[row['Latitud'], row['Longitud']] for _, row in x.iterrows()]
).reset_index(name='coordenadas')

# Formato requerido por HeatMapWithTime
heat_data = heatmap_data['coordenadas'].tolist()
time_index = heatmap_data['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S').tolist()

# Crear mapa base centrado en Bogotá
m = folium.Map(location=[4.65, -74.1], zoom_start=11)

# Agregar mapa de calor con slider de tiempo
HeatMapWithTime(heat_data, index=time_index, auto_play=False, max_opacity=0.6, radius=15,).add_to(m)

# Guardar HTML para visualización
map_path = "mapa_calor_accidentes.html"
m.save(map_path)

map_path

C:\Users\David Bonilla\AppData\Local\Temp\ipykernel_23296\2732625945.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  heatmap_data = gdf.groupby(['datetime']).apply(


'mapa_calor_accidentes.html'

In [17]:
from folium.plugins import MarkerCluster

# Cargar datos (ajusta la ruta o usa un DataFrame directo)
df = pd.read_csv("red_semaforos.csv")  # Debe tener columnas: lat, lon

# Crear un mapa base centrado en el promedio de las coordenadas
m = folium.Map(location=[df["Y"].mean(), df["X"].mean()], zoom_start=12)

# Crear un agrupador de marcadores
marker_cluster = MarkerCluster().add_to(m)

# Añadir puntos al cluster
for i, row in df.iterrows():
    folium.Marker(location=[row["Y"], row["X"]]).add_to(marker_cluster)

# Guardar como HTML
m.save("mapa_con_puntos.html")